In [1]:
cd  /content/drive/MyDrive/USP/Mestrado/Pesquisa/Artigos/Triple-VAE\:\ A\ Triple\ Variational\ Autoencoder\ to\ Represent\ Events\ in\ One-Class\ Event\ Detection/

/content/drive/MyDrive/USP/Mestrado/Pesquisa/Artigos/Triple-VAE: A Triple Variational Autoencoder to Represent Events in One-Class Event Detection


In [2]:
path = 'results_60/'

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

basepath = Path(path)
files_in_basepath = basepath.iterdir()

dic = {}

for item in files_in_basepath:
  if item.is_file():
    
    name = item.name.split('_')
    
    tam = len(name)
    
    dataset = name[0]
    
    preprocessing = name[1]
    
    df = pd.read_csv(path + item.name, sep=';')

    best_f1 = max(df['f1-score'])

    param_f1 = df[df['f1-score'] == best_f1]['Parameters'].iloc[0]

    best_pre = float(df[df['f1-score'] == best_f1]['precision'].iloc[0])

    brest_rev= float(df[df['f1-score'] == best_f1]['recall'].iloc[0])

    best_aucroc = max(df['auc_roc'])

    param_auc = df[df['auc_roc'] == best_aucroc]['Parameters'].iloc[0]

    best_acuracia = max(df['accuracy'])

    param_acc = df[df['accuracy'] == best_acuracia]['Parameters'].iloc[0]


    if dataset not in dic:
      dic[dataset] = {}
      dic[dataset][preprocessing] = pd.DataFrame(columns=['parameters_f1','parameters_auc','parameters_acc', 'precision',	'recall','f1-score','auc_roc', 'accuracy'])
    elif preprocessing not in dic[dataset]:
      dic[dataset][preprocessing] = pd.DataFrame(columns=['parameters_f1','parameters_auc','parameters_acc', 'precision', 'recall','f1-score','auc_roc', 'accuracy'])

    df_bests = dic[dataset][preprocessing]
    df_bests = df_bests.append({'parameters_f1' : param_f1,
                                'parameters_auc' : param_auc,
                                'parameters_acc' : param_acc,
                                'precision' : best_pre, 
                                'recall' : brest_rev, 
                                'f1-score' : best_f1, 
                                'auc_roc' : best_aucroc, 
                                'accuracy' : best_acuracia}, 
                               ignore_index=True)
    dic[dataset][preprocessing] = df_bests

In [4]:
def return_df(datasets, metric):
  l_column = ['Dataset']
  for cl1 in datasets['Covid'].keys():
    string = cl1
    l_column.append(string)
  
  df = pd.DataFrame(columns=l_column)
  for dataset in datasets.keys():
    dic_total = {'Dataset' : dataset}
    for prepro in datasets[dataset].keys():
      string = prepro 
      if metric == 'parameters_f1' or metric == 'parameters_auc' or metric == 'parameters_acc':
        dic_total[string] = datasets[dataset][prepro][metric][0]
      else:
        dic_total[string] = float(datasets[dataset][prepro][metric][0])

    df = df.append(dic_total, ignore_index=True)
  return df

In [7]:
return_df(dic,'f1-score')


,Dataset,VAE,Concat-DML-LL,TripleVAE,Lat-Long,VAE-DML-LL,Concatenate,BiVAE,AE-DML-LL,AE,DBERTML,Density
0,Earthquake,0.683501,0.667586,0.915810,0.668155,0.687113,0.667701,0.895865,0.679682,0.681350,0.866252,0.676439
1,Inflation,0.678777,0.670487,0.953439,0.670275,0.667344,0.670543,0.930440,0.685203,0.672288,0.809273,0.663900
2,Terrorism,0.668448,0.671196,0.937344,0.669022,0.670227,0.671882,0.908366,0.666836,0.667993,0.767797,0.708839
3,Racism,0.675219,0.663313,0.964416,0.691277,0.670935,0.663313,0.960528,0.674306,0.675404,0.824531,0.685330
4,Agriculture,0.664633,0.666851,0.978697,0.666113,0.662103,0.666851,0.969712,0.665799,0.665397,0.756983,0.688913
5,Corruption,0.652032,0.693764,0.957611,0.692315,0.639770,0.693764,0.939468,0.653598,0.640844,0.727793,0.681325
6,Covid,0.736229,0.688087,0.946491,0.663900,0.740880,0.687964,0.951903,0.742527,0.741371,0.840736,0.754052
7,Immigration,0.689388,0.670156,0.950327,0.674269,0.697297,0.673929,0.942422,0.698798,0.691728,0.748046,0.690767
8,Tsunami,0.673229,0.663568,0.917924,0.663900,0.669114,0.663568,0.897326,0.670570,0.671622,0.880613,0.727326
9,War,0.683730,0.675218,0.779758,0.671609,0.681529,0.675218,0.721105,0.678037,0.679684,0.662552,0.730011


In [ ]:
for metric in ['f1-score','accuracy','auc_roc']:
  return_df(dic,metric).to_csv('result_60_' + metric + '.csv', sep=';')